## Sequential Hypothesis Testing for AI Auditing
### Author: Martín Anaya

In this notebook, we design controlled experiments using synthetic data and simulated black-box models. The purpose of these experiments is to evaluate the Sequential Probability Ratio Test (SPRT) under reproducible and well-defined conditions as an auditing tool.

### Library Imports
We begin by loading the libraries that we will use in this experiment.

In [ ]:
# Utilities
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical libraries
from scipy.stats import norm

### Simulation of black-box outputs

In this framework, the black-box model is represented as a probabilistic mapping from pre-defined biased probabilities to system outcomes. More precisely, we define a probabilistic setup in which individuals are assigned a likelihood of being selected or rejected depending on an attribute noted _suitability_. This attribute allows us to find the true positive and true negative outcomes of the model. By controlling these probabilities, we are able to introduce systematic bias into the model while maintaining transparency over the process of data generation.


To operationalise the model, we generate two sets of probabilities:

- Suitability rates: Defines the probability of an individual belonging to the "suitable" group. We assume both groups to have the same likelihood of being truly suitable.
    
    $$\mathbb{P}(\text{suitable } | \text{ Group = 1}) = 0.8 \qquad \mathbb{P}(\text{suitable } | \text{ Group = 2}) = 0.8$$

- Bias rates: Defines the probability that the model assigns an individual to the positive class depending on both their group and their true suitability. 
    $$\mathbb{P}(\text{positive } | \text{ Group = 1, suitable}) = 0.8 \qquad \mathbb{P}(\text{positive } | \text{ Group = 1, unsuitable}) = 0.25$$
    $$\mathbb{P}(\text{positive } | \text{ Group = 2, suitable}) = 0.5 \qquad \mathbb{P}(\text{positive } | \text{ Group = 2, unsuitable}) = 0.1$$


This setup ensures that, despite that both groups contain the same proportion of qualified individuals, the model systematically favours Group 1 over Group 2.

In [ ]:
# Function to simulate the black-box model
def black_box_outputs(num_group1, num_group2, suitability_rates, bias_rates, seed = 42):
    """
        Parameters: 
        num_group1: number of individuals from group 1
        num_group2: number of individuals from group 2

        suitability_rates: Probabilities of individuals from each group to belong to the suitable group
        bias_rates: Probabilities of the model assigning an individual to the positive class depending on their group and suitability

        returns:
        dataframe with the outputs of the system
        
    """
    random_generator = np.random.default_rng(seed)

    # Probabilistic suitability rate
    num_group1_suitable = sum(random_generator.random(num_group1) < suitability_rates['group1_prob_suitable'])
    num_group1_unsuitable = num_group1 - num_group1_suitable

    num_group2_suitable = sum(random_generator.random(num_group2) < suitability_rates['group2_prob_suitable'])
    num_group2_unsuitable = num_group2 - num_group2_suitable

    # Probabilistic bias rate
    positives_group1_suitable = sum(random_generator.random(num_group1_suitable) < bias_rates['group1_suitable'])
    positives_group1_unsuitable = sum(random_generator.random(num_group1_unsuitable) < bias_rates['group1_unsuitable'])
    positives_group2_suitable = sum(random_generator.random(num_group2_suitable) < bias_rates['group2_suitable'])
    positives_group2_unsuitable = sum(random_generator.random(num_group2_unsuitable) < bias_rates['group2_unsuitable'])

    negatives_group1_suitable = num_group1_suitable - positives_group1_suitable
    negatives_group1_unsuitable = num_group1_unsuitable - positives_group1_unsuitable
    negatives_group2_suitable = num_group2_suitable - positives_group2_suitable
    negatives_group2_unsuitable = num_group2_unsuitable - positives_group2_unsuitable

    # Outputs
    outputs = [
        ['G1_suit', positives_group1_suitable, negatives_group1_suitable],
        ['G1_unsuit', positives_group1_unsuitable, negatives_group1_unsuitable],
        ['G2_suit', positives_group2_suitable, negatives_group2_suitable],
        ['G2_unsuit', positives_group2_unsuitable, negatives_group2_unsuitable]
    ]

    df = pd.DataFrame(outputs, columns=['group', 'Positive class', 'Negative class'])
    return df

# Obtain a dataframe with the group, prediction and true label of each individual. Those will be the samples that we will use in the test
def get_individual_df(df):
    data = []
    # Expand each row in individuals
    for _, row in df.iterrows():
        group_name = row['group']
        group = group_name[:2]                      # 'G1' or 'G2'
        suitability = 1 if '_suit' in group_name else 0
        
        # Add positive individuals
        data.extend([{
            'group': group,
            'suitability': suitability,
            'prediction': 1
        }] * row['Positive class'])
    
        # Add negative individuals
        data.extend([{
            'group': group,
            'suitability': suitability,
            'prediction': 0
            }] * row['Negative class'])

    return pd.DataFrame(data)
    

#### Bias rate
In the following cell, we define the probabilities of our model to generate each of the outputs.

In [ ]:
simulation_suitability_rates = {'group1_prob_suitable' : 0.8, 
                      'group2_prob_suitable' : 0.8}

simulation_bias_rates = {'group1_suitable' : 0.8,
                            'group1_unsuitable' : 0.25,
                            'group2_suitable' : 0.5,
                            'group2_unsuitable' : 0.1}

alpha = 0.05
beta = 0.2

Now we can generate the outputs of the system.

In [ ]:
df_outputs = black_box_outputs(1000,1000,simulation_suitability_rates,simulation_bias_rates, seed = 42)

In [ ]:
# Distribution of outcomes
print(df_outputs)

#### Output Visualization


In [ ]:
def pie_charts_outputs_global(df):
    df_grouped = df.groupby(df['group'].str.slice(0, 2))[['Positive class', 'Negative class']].sum().reset_index()

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle('Model predictions by group (G1 / G2)', fontsize=16)
    
    for i, ax in enumerate(axs.flat):
        group = df_grouped.loc[i, 'group']
        hired = df_grouped.loc[i, 'Positive class']
        not_hired = df_grouped.loc[i, 'Negative class']
        ax.pie([hired, not_hired],
               labels=['Positive class', 'Negative class'],
               autopct='%1.1f%%',
               colors=sns.color_palette("pastel"),
               startangle=90)
        ax.set_title(group)
    
    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.show()

def pie_charts_outputs_subgroups(df):
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    fig.suptitle('Model predictions by subgroup', fontsize=16)
    
    for i, ax in enumerate(axs.flat):
        group = df.loc[i, 'group']
        hired = df.loc[i, 'Positive class']
        not_hired = df.loc[i, 'Negative class']
        ax.pie([hired, not_hired],
               labels=['Positive class', 'Negative class'],
               autopct='%1.1f%%',
               colors=sns.color_palette("pastel")[2:],
               startangle=90)
        ax.set_title(group)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()



The following graphs illustrate the global distribution of the classifier's outcomes, providing a general idea of how the black-box model assigns positive and negative decisions. This representation manifests the noteworthy bias of our black-box model.

In [ ]:
pie_charts_outputs_global(df_outputs)

In [ ]:
pie_charts_outputs_subgroups(df_outputs)

In [ ]:
# Transform the dataframe into the correct format for the SPRT
individual_df = get_individual_df(df_outputs)

In [ ]:
print(individual_df)

It is also interesting to see the average output of the system for each subgroup.

In [ ]:
# Mean output per subgroup
individual_df.groupby(['group', 'suitability'])['prediction'].mean()

### Fairness metrics
In the following block, we define the fairness criteria that we will consider in the test.

In [ ]:
def sprt_statistical_parity(group1, group2, alpha=0.05, beta=0.2, delta=0.1):
    """
    Statistical parity for SPRT
    """
    # Number of individuals of each group
    n1 = len(group1)
    n2 = len(group2)

    s1 = group1['prediction'].sum()  # Number of predicted positives in group 1
    s2 = group2['prediction'].sum()  # Number of predicted positives in group 2

    # Observed proportions
    p1_hat = s1 / n1
    p2_hat = s2 / n2

    # Define the hypotheses. Consider p1 > p2 to save runtime. Otherwise, just change the order of the variables.
    p0 = (p1_hat + p2_hat) / 2
    p1 = p0 + delta/2
    p2 = p0 - delta/2

    # Likelihood ratio
    L0 = (p0**s1 * (1 - p0)**(n1 - s1)) * (p0**s2 * (1 - p0)**(n2 - s2))
    L1 = (p1**s1 * (1 - p1)**(n1 - s1)) * (p2**s2 * (1 - p2)**(n2 - s2))
    lr = L1 / L0 if L0 > 0 else np.inf

    # log likelihood approach
    #log_L0 = s1 * np.log(p0) + (n1 - s1) * np.log(1 - p0) + s2 * np.log(p0) + (n2 - s2) * np.log(1 - p0)
    #log_L1 = s1 * np.log(p1) + (n1 - s1) * np.log(1 - p1) + s2 * np.log(p2) + (n2 - s2) * np.log(1 - p2)
    #log_lr = log_L1 - log_L0
    #lr = np.exp(log_lr)
    
    # Thresholds
    A = (1 - beta) / alpha
    B = beta / (1 - alpha)
    
    # Decision
    if lr >= A:
        return "Reject H0 (difference detected)", lr
    elif lr <= B:
        return "Accept H0 (no difference)", lr
    else:
        return "Continue sampling", lr


def sprt_equal_opportunity(group1, group2, alpha=0.05, beta=0.2, delta=0.1):
    """
    Equal Opportunity fairness criterion for SPRT.
    Compares true positive rates (TPR) between two groups.
    """

    # Filter only the truly positive individuals (Y=1)
    g1_pos = group1[group1['suitability'] == 1]
    g2_pos = group2[group2['suitability'] == 1]
    
    n1 = len(g1_pos)
    n2 = len(g2_pos)

    # Need at least one sample of each group.
    if n1 == 0 or n2 == 0:
        return "Continue sampling" , np.nan

    s1 = g1_pos['prediction'].sum()  # Number of predicted positives in group 1
    s2 = g2_pos['prediction'].sum()  # Number of predicted positives in group 2

    # Observed TPRs
    p1_hat = s1 / n1
    p2_hat = s2 / n2

    # Define the hypotheses. Consider p1 > p2 to save runtime. Otherwise, just change the order of the variables.
    p0 = (p1_hat + p2_hat) / 2  # estimador común bajo H0
    p1 = p0 + delta/2
    p2 = p0 - delta/2

    # Likelihood ratio
    L0 = (p0**s1 * (1 - p0)**(n1 - s1)) * (p0**s2 * (1 - p0)**(n2 - s2))
    L1 = (p1**s1 * (1 - p1)**(n1 - s1)) * (p2**s2 * (1 - p2)**(n2 - s2))
    lr = L1 / L0 if L0 > 0 else np.inf

    # Thresholds
    A = (1 - beta) / alpha
    B = beta / (1 - alpha)

    # Decision
    if lr >= A:
        return "Reject H0 (difference detected)", lr
    elif lr <= B:
        return "Accept H0 (no difference)", lr
    else:
        return "Continue sampling", lr


def sprt_equalized_odds(group1, group2, alpha=0.05, beta=0.2, delta=0.1):
    """
    Equalized Odds for SPRT
      - TPR parity: P(ŷ=1 | Y=1, A=a) vs P(ŷ=1 | Y=1, A=b)
      - FPR parity: P(ŷ=1 | Y=0, A=a) vs P(ŷ=1 | Y=0, A=b)
    """


    # Filter only the truly positive individuals (Y=1)
    g1_pos = group1[group1['suitability'] == 1]
    g2_pos = group2[group2['suitability'] == 1]
    
    n1 = len(g1_pos)
    n2 = len(g2_pos)

    # Filter only the truly negative individuals (Y=1)
    g1_neg = group1[group1['suitability'] == 0]
    g2_neg = group2[group2['suitability'] == 0]
    
    n1_neg = len(g1_neg)
    n2_neg = len(g2_neg)

    # Need at least one sample from each group
    if n1 == 0 or n2 == 0 or n1_neg == 0 or n2_neg == 0:
        return "Continue sampling" , np.nan

    s1 = g1_pos['prediction'].sum() 
    s2 = g2_pos['prediction'].sum() 

    # Observed TPRs
    p1_hat = s1 / n1
    p2_hat = s2 / n2

    # Define the hypotheses. Consider p1 > p2 to save runtime. Otherwise, just change the order of the variables.
    p0 = (p1_hat + p2_hat) / 2 
    p1 = p0 + delta/2
    p2 = p0 - delta/2

    # Likelihood ratio
    L0 = (p0**s1 * (1 - p0)**(n1 - s1)) * (p0**s2 * (1 - p0)**(n2 - s2))
    L1 = (p1**s1 * (1 - p1)**(n1 - s1)) * (p2**s2 * (1 - p2)**(n2 - s2))
    lr = L1 / L0 if L0 > 0 else np.inf

    # Thresholds
    A = (1 - beta) / alpha
    B = beta / (1 - alpha)

    # Decisión
    if lr >= A:
        tpr, lr_tpr = "Reject H0 (difference detected)", lr
    elif lr <= B:
        tpr, lr_tpr = "Accept H0 (no difference)", lr
    else:
        tpr, lr_tpr = "Continue sampling", lr



    s1_neg = g1_neg['prediction'].sum()  # contratados en grupo 1
    s2_neg = g2_neg['prediction'].sum()  # contratados en grupo 2

    # Observed TPRs
    p1_hat_neg = s1_neg / n1_neg
    p2_hat_neg = s2_neg / n2_neg

    p0_neg = (p1_hat_neg + p2_hat_neg) / 2  
    p1_neg = p0_neg + delta/2
    p2_neg = p0_neg - delta/2

    # Likelihood ratio
    L0_neg = (p0_neg**s1_neg * (1 - p0_neg)**(n1_neg - s1_neg)) * (p0_neg**s2_neg * (1 - p0_neg)**(n2_neg - s2_neg))
    L1_neg = (p1_neg**s1_neg * (1 - p1_neg)**(n1_neg - s1_neg)) * (p2_neg**s2_neg * (1 - p2_neg)**(n2_neg - s2_neg))
    lr_neg = L1_neg / L0_neg if L0_neg > 0 else np.inf

    # Decision
    if lr_neg >= A:
        fpr, lr_fpr = "Reject H0 (difference detected)", lr_neg
    elif lr_neg <= B:
        fpr, lr_fpr = "Accept H0 (no difference)", lr_neg
    else:
        fpr, lr_fpr = "Continue sampling", lr_neg

# --- Decisión global EO
    if tpr.startswith("Accept") and fpr.startswith("Accept"):
        global_decision = "Accept H0 (no difference)"
    elif tpr.startswith("Reject") or fpr.startswith("Reject"):
        global_decision = "Reject H0 (difference detected)"
    else:
        global_decision = "Continue sampling"

    return global_decision , np.max([lr_tpr, lr_fpr])


### Sequential Probability Ratio Test loop
Here we implement the main loop for the sequential test.

In [ ]:
def run_test(df, batch_size=10, max_steps=10, alpha = 0.05, beta = 0.2, delta=0.1, criterion="SP"):
    # Create sample pool
    pool = individual_df.copy()
    pool = pool.sample(frac=1).reset_index(drop=True) # random order

    results = []

    # Add one individual from each group to avoid divisions by 0 in the first batch.
    g1_index = pool[pool['group'] == 'G1'].index[0]
    g2_index = pool[pool['group'] == 'G2'].index[0]
    
    # Evidence set
    accumulated = pool.loc[[g1_index, g2_index]]
    
    pool = pool.drop([g1_index, g2_index])

    for step in range(max_steps):
        if len(pool) < batch_size:
            print("No hay suficientes datos para continuar.")
            break

        batch = pool.iloc[:batch_size]
        pool = pool.iloc[batch_size:]
        
        accumulated = pd.concat([accumulated, batch], ignore_index=True)

        g1 = accumulated[accumulated['group'] == 'G1']
        g2 = accumulated[accumulated['group'] == 'G2']
        
        # Choose test depending on criterion
        if criterion == "SP":
            decision, lr = sprt_statistical_parity(g1, g2, alpha = alpha, beta = beta, delta=delta)
        elif criterion == "EOP":
            decision, lr = sprt_equal_opportunity(g1, g2, alpha = alpha, beta = beta, delta=delta)
        elif criterion == "EO":
            decision, lr = sprt_equalized_odds(g1, g2, alpha = alpha/2, beta = beta, delta=delta) # Bonferroni correction
        else:
            raise ValueError("criterion must be 'SP' or 'EO' or 'EOP'")

        
        #print(f"Step {step+1}: {decision}, LR={lr:.4f}")
        results.append((step+1, decision, lr))
        
        if decision != "Continue sampling":
            break

    return results


### Neyman-Pearson fixed sample size
We use the following formula to calculate the sample size of a proportion tests under the Neyman-Pearson framework:

$$n = \frac{(Z_{1-\beta} + Z_{1-\frac{\alpha}{2}})^2 (\pi_A(1-\pi_A)+\pi_B(1-\pi_B)) }{(\pi_B - \pi_A)^2}$$

where
- $n$ is the required number of observations in each of the two groups (therefore the total sample size is $2n$).
- $Z_{1-x}$ is the value from the standardised normal distribution for which the probability of exceeding it is $x$. 
- $\pi_A$ and $\pi_B$ are the anticipated probabilities of event in groups A and B. 
- $\alpha$ is the type I error rate, and $\beta$ the type II error rate.

In [ ]:
def sample_size_neyman_pearson(suitability, bias, alpha=0.05, beta=0.2):

    #Pi
    pi_A, pi_B = compute_pi(suitability, bias)
    
    Z_alpha = norm.ppf(1 - alpha/2)   # para test bilateral
    #Z_alpha = norm.ppf(1 - alpha)   # para test one-sided
    Z_beta  = norm.ppf(1 - beta)         # 1 - beta

    delta = abs(pi_B - pi_A)

    var = pi_A * (1 - pi_A) + pi_B * (1 - pi_B)

    n = ((Z_alpha + Z_beta)**2 * var) / (delta**2)

    # Formula for Sample size per group, therefore the total is x2
    total = int(np.ceil(n)) * 2
    
    return total


def sample_size_neyman_pearson_criterion(suitability, bias, alpha=0.05, beta=0.2, criterion = 'SP'):
    
    #Pi
    pi_A, pi_B = compute_pi_criterion(suitability, bias, criterion)
    
    Z_alpha = norm.ppf(1 - alpha/2)   
    Z_beta  = norm.ppf(1 - beta)         # 1 - beta

    delta = abs(pi_B - pi_A)

    var = pi_A * (1 - pi_A) + pi_B * (1 - pi_B)

    n = ((Z_alpha + Z_beta)**2 * var) / (delta**2)

    # Formula for Sample size per group, therefore the total is x2
    total = int(np.ceil(n)) * 2
    
    return total

def compute_pi(suitability_rates, bias_rates):
    pi1 = suitability_rates['group1_prob_suitable'] * bias_rates['group1_suitable'] + (1-suitability_rates['group1_prob_suitable']) * bias_rates['group1_unsuitable']
    pi2 = suitability_rates['group2_prob_suitable'] * bias_rates['group2_suitable'] + (1-suitability_rates['group2_prob_suitable']) * bias_rates['group2_unsuitable']
    
    return pi1, pi2


def compute_pi_criterion(suitability_rates, bias_rates, criterion):
    # Choose test depending on criterion
    if criterion == "SP":
        pi1 = suitability_rates['group1_prob_suitable'] * bias_rates['group1_suitable'] + (1-suitability_rates['group1_prob_suitable']) * bias_rates['group1_unsuitable']
        pi2 = suitability_rates['group2_prob_suitable'] * bias_rates['group2_suitable'] + (1-suitability_rates['group2_prob_suitable']) * bias_rates['group2_unsuitable']
    
    elif criterion == "EOP":
        pi1 = bias_rates['group1_suitable']
        pi2 = bias_rates['group2_suitable']
    
    else:
        raise ValueError("criterion must be 'SP' or 'EOP'")  
        
    return pi1, pi2


def sample_size_neyman_pearson_equalized_odds(suitability, bias, alpha=0.05, beta=0.2, show_groups=False):

    #Pi
    pi_A_tpr = bias['group1_suitable']
    pi_B_tpr =  bias['group2_suitable']


    pi_A_fpr = bias['group1_unsuitable']
    pi_B_fpr = bias['group2_unsuitable']

    alpha_prime = alpha/2 # Corrección de Bonferroni
    
    Z_alpha = norm.ppf(1 - alpha_prime/2)  
    Z_beta  = norm.ppf(1 - beta)         # 1 - beta

    delta_tpr = abs(pi_B_tpr - pi_A_tpr)
    delta_fpr = abs(pi_B_fpr - pi_A_fpr)

    var_tpr = pi_A_tpr * (1 - pi_A_tpr) + pi_B_tpr * (1 - pi_B_tpr)
    var_fpr = pi_A_fpr * (1 - pi_A_fpr) + pi_B_fpr * (1 - pi_B_fpr)

    n_tpr = ((Z_alpha + Z_beta)**2 * var_tpr) / (delta_tpr**2)
    n_fpr = ((Z_alpha + Z_beta)**2 * var_fpr) / (delta_fpr**2)

    if (show_groups):
        print("Sample size TPR: ", int(np.ceil(n_tpr))*2, "\nSample Size FPR: " , int(np.ceil(n_fpr))*2)

    
    n_max = max(n_tpr, n_fpr)
    
    # Formula for Sample size per group, therefore the total is x2
    total = int(np.ceil(n_max)) * 2
    
    return total

In [ ]:
n_theoretical_sp = sample_size_neyman_pearson_criterion(simulation_suitability_rates, simulation_bias_rates, alpha=0.05, beta=0.2 , criterion= 'SP')

print("Tamaño de muestra necesario:", n_theoretical_sp, "en total (Statistical Parity).")

In [ ]:
n_theoretical_eo = sample_size_neyman_pearson_criterion(simulation_suitability_rates, simulation_bias_rates, alpha=0.05, beta=0.2 , criterion= 'EOP')

print("Tamaño de muestra necesario:", n_theoretical_eo, "en total (Equal Opportunity).")

In [ ]:
n_theoretical_eodds = sample_size_neyman_pearson_equalized_odds(simulation_suitability_rates, simulation_bias_rates, alpha=0.05, beta=0.2, show_groups=True)

print("Tamaño de muestra necesario:", n_theoretical_eodds, "en total (Equalized Odds).")

### Experiments
Remember that the sprt loop starts with 2 initial samples. Then, the total number of samples used are *2 + num_steps * batch_size*.

In [ ]:
batch_size = 1
max_steps = 2000
delta = 0.2
num_simulations = 10

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = 0.05, beta = 0.2, delta = delta, criterion = 'SP')
    print(">>> Simulation", i+1, ":")
    print(res[-1])
    print("Samples used: ", res[-1][0] * batch_size +2)
    print("\n")

#### Likelihood ratio Visualization

In [ ]:
alpha = 0.05
beta = 0.2
delta = 0.1

res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')
res = pd.DataFrame(res)

likelihood_ratios = res.iloc[:,2]


x_axis = np.arange(1, len(likelihood_ratios) + 1) * batch_size
A = (1 - beta) / alpha
B = beta / (1 - alpha)



# Plot
plt.figure(figsize=(10,6))

# Y limits
center = (A + B) / 2
half_range = (A - B) / 2 
y_min = center - (2.25*half_range)
y_max = center + (2.25*half_range)

plt.ylim(y_min, y_max)

# Background
plt.axhspan(A, y_max, color="mistyrose", alpha=0.5, label="Reject H0 region")
plt.axhspan(y_min, B, color="lightblue", alpha=0.5, label="Accept H0 region")
plt.axhspan(B, A, color="whitesmoke", alpha=0.5, label="Continue sampling")

# LR curve
plt.plot(x_axis, likelihood_ratios, marker='o', color="black", label="LR Evolution", linewidth=2)

# Threshold lines
plt.axhline(A, color="red", linestyle="--", linewidth=1.5)
plt.axhline(B, color="blue", linestyle="--", linewidth=1.5)

# Labels and title
plt.xlabel("Total number of evaluated samples", fontsize=12)
plt.ylabel("Cumulative Likelihood Ratio", fontsize=12)
plt.title("SPRT Decision Regions", fontsize=14, weight="bold")

plt.legend(loc="best")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


#### Batch size Values
In this section, we try different batch sizes values to check its effect on the SPRT decision.

In [ ]:
batch_size = 10
max_steps = 2000
delta = 0.2
num_simulations = 100

alpha = 0.05
beta = 0.2


resultados_simulaciones = pd.DataFrame()
resultados_globales = pd.DataFrame(columns = ['Batch size', 'Mean samples', 'Diff detected'])

for j in range(100):
    resultados_simulaciones = pd.DataFrame()
    for i in range(num_simulations): 
        res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
        res = pd.DataFrame(res).transpose()
    
        col = res.columns[1]
    
        res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
        res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
        res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
        resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

    resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

    resultados_globales.loc[len(resultados_globales)] = [batch_size, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]
    batch_size = batch_size + 10


In [ ]:
print(resultados_globales[0:50])

Regarding the choice of batch size, we remark that it can influence the early trajectory of the first estimates, but its overall impact on the final required sample size and test decision is limited. Using a larger batch size will help mitigate variability and the effect of initial random fluctuations at the start of the experiment (we will see that the average sample size of failed tests is considerably small than the sample size of successful ones). However, in our study we will intentionally maintain a small batch size (each batch consists of two observations) to precisely identify the minimum number of observations required by the SPRT, ensuring the estimates are as accurate as possible.

#### Delta Values
A careful choice of the threshold parameter $\delta$ is essential, as it balances test sensitivity and sample efficiency. On one hand, smaller values of $\delta$ increase the test's ability to detect subtle biases, but will typically require larger sample sizes. On the other hand, larger $\delta$ values reduce the number of samples needed, but may miss minor biases.

In [ ]:
batch_size = 2
max_steps = 2000
num_simulations = 1000

alpha = 0.05
beta = 0.2


resultados_simulaciones = pd.DataFrame()
resultados_globales = pd.DataFrame(columns = ['Delta', 'Mean samples', 'Diff detected'])

# Small delta
delta = 0.05
for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Increase delta
delta = 0.1
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Increase delta
delta = 0.15
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Moderate delta
delta = 0.20
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Moderate delta
delta = 0.25
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Moderate delta
delta = 0.30
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Large delta
delta = 0.35
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


# Large delta
delta = 0.4
resultados_simulaciones = pd.DataFrame()

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [delta, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("\nDelta: ", delta,"\nNum simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())


In [ ]:
print(resultados_globales)

Let's visualize the results:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,6))
resultados_globales["Delta"] = resultados_globales["Delta"].astype(str) #Convertir a categoría
resultados_globales["Diff detected"] = resultados_globales["Diff detected"] *100 / num_simulations #Convertir a %


# Mean samples per Delta
axes[0].bar(resultados_globales["Delta"], resultados_globales["Mean samples"], color="skyblue", edgecolor="black")
axes[0].set_title("Average Sample Size per Delta", fontsize=14, weight="bold")
axes[0].set_xlabel("Delta", fontsize=12)
axes[0].set_ylabel("Mean Sample Size", fontsize=12)
axes[0].grid(axis="y", linestyle="--", alpha=0.6)

# Diffs detected per Delta
axes[1].bar(resultados_globales["Delta"], resultados_globales["Diff detected"], color="lightcoral", edgecolor="black")
axes[1].set_title("Differences Detected per Delta", fontsize=14, weight="bold")
axes[1].set_xlabel("Delta", fontsize=12)
axes[1].set_ylabel("Differences Detected (%)", fontsize=12)
axes[1].grid(axis="y", linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()

### Simulations
In this section, we present a series of experiments designed to evaluate the performance of the SPRT as an auditing tool. Our empirical evaluation focuses on quantifying the number of observations required to detect bias under different fairness metrics. For all experiments here, we used a $\delta$ value of 0.2. Ideally, we could have chosen $\delta = 0.3$, since we know the bias rates of our black-box model. However, we selected a more conservative choice to better demonstrate the behaviour of our audit framework under conditions where the bias magnitude is unknown, ensuring both a meaningful bias detection and realistic benchmark. For the results presented in the following subsections, ten thousand simulations were run for each metric.
#### Statistical Parity

In [ ]:
batch_size = 2
max_steps = 2000
delta = 0.2
num_simulations = 10000

alpha = 0.05
beta = 0.2


resultados_simulaciones = pd.DataFrame()
resultados_globales = pd.DataFrame(columns = ['Batch size', 'Mean samples', 'Diff detected'])


for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'SP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones = pd.concat([resultados_simulaciones, res], ignore_index=True)

resultados_simulaciones.columns = ["Num_samples", "Difference Detected"]

resultados_globales.loc[len(resultados_globales)] = [batch_size, resultados_simulaciones['Num_samples'].mean(), resultados_simulaciones['Difference Detected'].sum()]

print("Num simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones['Difference Detected'].sum())

In [ ]:
resultados_simulaciones.loc[:,"Num_samples"]

In [ ]:
print("Num experiments with less samples than n_theoretical:", len(resultados_simulaciones[resultados_simulaciones["Num_samples"]<=n_theoretical_sp])) 
print("Num of correct experiments with less samples than n_theoretical:", len(resultados_simulaciones[(resultados_simulaciones["Num_samples"]<=n_theoretical_sp) & (resultados_simulaciones["Difference Detected"] == 1)]))

print(f"Average sample size of correct experiments: {resultados_simulaciones[resultados_simulaciones['Difference Detected'] == 1]['Num_samples'].mean():.2f}")
print(f"Average sample size of failed experiments: {resultados_simulaciones[resultados_simulaciones['Difference Detected'] == 0]['Num_samples'].mean():.2f}")

In [ ]:
plt.figure(figsize=(7, 4))
plt.hist(resultados_simulaciones.loc[:,"Num_samples"], bins=30, color="skyblue", edgecolor="black", alpha=0.7, label="Simulations")
plt.axvline(n_theoretical_sp, color="red", linestyle="--", linewidth=2, label="Neyman–Pearson expected")

plt.title("Sample size distribution - Statistical Parity")
plt.xlabel("Sample size")
plt.ylabel("Frecuency")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

#### Equal Opportunity

In [ ]:
resultados_simulaciones_eop = pd.DataFrame()
resultados_globales_eop = pd.DataFrame(columns = ['Batch size', 'Mean samples', 'Diff detected'])


for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'EOP')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones_eop = pd.concat([resultados_simulaciones_eop, res], ignore_index=True)

resultados_simulaciones_eop.columns = ["Num_samples", "Difference Detected"]

resultados_globales_eop.loc[len(resultados_globales_eop)] = [batch_size, resultados_simulaciones_eop['Num_samples'].mean(), resultados_simulaciones_eop['Difference Detected'].sum()]

print("Num simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones_eop['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones_eop['Difference Detected'].sum())

In [ ]:
print("Num experiments with less samples than n_theoretical:", len(resultados_simulaciones_eop[resultados_simulaciones_eop["Num_samples"]<=n_theoretical_eo])) 
print("Num of correct experiments with less samples than n_theoretical:", len(resultados_simulaciones_eop[(resultados_simulaciones_eop["Num_samples"]<=n_theoretical_eo) & (resultados_simulaciones_eop["Difference Detected"] == 1)]))


print(f"Average sample size of correct experiments: {resultados_simulaciones_eop[resultados_simulaciones_eop['Difference Detected'] == 1]['Num_samples'].mean():.2f}")
print(f"Average sample size of failed experiments: {resultados_simulaciones_eop[resultados_simulaciones_eop['Difference Detected'] == 0]['Num_samples'].mean():.2f}")

In [ ]:
n_theoretical_eop = sample_size_neyman_pearson_criterion(simulation_suitability_rates, simulation_bias_rates, alpha=0.05, beta=0.2 , criterion= 'EOP')
n_theoretical_eop_corrected = n_theoretical_eop/simulation_suitability_rates['group1_prob_suitable']

plt.figure(figsize=(7, 4))
plt.hist(resultados_simulaciones_eop.loc[:,"Num_samples"], bins=30, color="skyblue", edgecolor="black", alpha=0.7, label="Simulations")
plt.axvline(n_theoretical_eop, color="red", linestyle="--", linewidth=2, label="Neyman–Pearson expected")
plt.axvline(n_theoretical_eop_corrected, color="green", linestyle="--", linewidth=2, label="Neyman–Pearson corrected")


plt.title("Sample size distribution - Equal Opportunity")
plt.xlabel("Sample size")
plt.ylabel("Frecuency")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

#### Equalized odds

In [ ]:
resultados_simulaciones_eodds = pd.DataFrame()
resultados_globales_eodds = pd.DataFrame(columns = ['Batch size', 'Mean samples', 'Diff detected'])

for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'EO')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    res.loc[res[col] == 'Continue sampling', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones_eodds = pd.concat([resultados_simulaciones_eodds, res], ignore_index=True)

resultados_simulaciones_eodds.columns = ["Num_samples", "Difference Detected"]

resultados_simulaciones_eodds
resultados_globales_eodds.loc[len(resultados_globales_eodds)] = [batch_size, resultados_simulaciones_eodds['Num_samples'].mean(), resultados_simulaciones_eodds['Difference Detected'].sum()]

print("Num simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones_eodds['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones_eodds['Difference Detected'].sum())

In [ ]:
print("Num experiments with less samples than n_theoretical:", len(resultados_simulaciones_eodds[resultados_simulaciones_eodds["Num_samples"]<=n_theoretical_eodds])) 
print("Num of correct experiments with less samples than n_theoretical:", len(resultados_simulaciones_eodds[(resultados_simulaciones_eodds["Num_samples"]<=n_theoretical_eodds) & (resultados_simulaciones_eodds["Difference Detected"] == 1)]))


print(f"Average sample size of correct experiments: {resultados_simulaciones_eodds[resultados_simulaciones_eodds['Difference Detected'] == 1]['Num_samples'].mean():.2f}")
print(f"Average sample size of failed experiments: {resultados_simulaciones_eodds[resultados_simulaciones_eodds['Difference Detected'] == 0]['Num_samples'].mean():.2f}")

In [ ]:
n_theoretical_eodds = sample_size_neyman_pearson_equalized_odds(simulation_suitability_rates, simulation_bias_rates, alpha=0.05, beta=0.2 )
n_theoretical_eodds_corrected = n_theoretical_eodds/simulation_suitability_rates['group1_prob_suitable']

plt.figure(figsize=(7, 4))
plt.hist(resultados_simulaciones_eodds.loc[:,"Num_samples"], bins=30, color="skyblue", edgecolor="black", alpha=0.7, label="Simulations")
plt.axvline(n_theoretical_eodds, color="red", linestyle="--", linewidth=2, label="Neyman–Pearson expected")

plt.title("Sample size distribution - Equalized Odds")
plt.xlabel("Sample size")
plt.ylabel("Frecuency")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

#### Fair system example

In [ ]:
simulation_suitability_rates = {'group1_prob_suitable' : 0.8, 
                      'group2_prob_suitable' : 0.8}

simulation_bias_rates = {'group1_suitable' : 0.9,
                            'group1_unsuitable' : 0.2,
                            'group2_suitable' : 0.9,
                            'group2_unsuitable' : 0.2}

df_outputs = black_box_outputs(1000,1000,simulation_suitability_rates, simulation_bias_rates, seed = 42)

individual_df = get_individual_df(df_outputs)

In [ ]:
individual_df.groupby(['group', 'suitability'])['prediction'].mean()

In [ ]:
resultados_simulaciones_eodds = pd.DataFrame()
resultados_globales_eodds = pd.DataFrame(columns = ['Batch size', 'Mean samples', 'Diff detected'])

num_simulations = 1000
for i in range(num_simulations): 
    res = run_test(df = individual_df, batch_size = batch_size, max_steps = max_steps, alpha = alpha, beta = beta, delta = delta, criterion = 'EO')[-1][0:2]
    res = pd.DataFrame(res).transpose()
    
    col = res.columns[1]
    
    res.loc[res[col] == 'Reject H0 (difference detected)', col] = 1
    res.loc[res[col] == 'Accept H0 (no difference)', col] = 0
    res.loc[res[col] == 'Continue sampling', col] = 0
    
    res.iloc[:,0] = res.iloc[:,0] * batch_size + 2
        
    resultados_simulaciones_eodds = pd.concat([resultados_simulaciones_eodds, res], ignore_index=True)

resultados_simulaciones_eodds.columns = ["Num_samples", "Difference Detected"]

print("Num simulaciones:" , num_simulations, "\nMean sample size: ", resultados_simulaciones_eodds['Num_samples'].mean(), "\nDiffs detected: ", resultados_simulaciones_eodds['Difference Detected'].sum())

In [ ]:
print(f"Average sample size of correct experiments: {resultados_simulaciones_eodds[resultados_simulaciones_eodds['Difference Detected'] == 1]['Num_samples'].mean():.2f}")
print(f"Average sample size of failed experiments: {resultados_simulaciones_eodds[resultados_simulaciones_eodds['Difference Detected'] == 0]['Num_samples'].mean():.2f}")

In [ ]:
plt.figure(figsize=(7, 4))
plt.hist(resultados_simulaciones_eodds.loc[:,"Num_samples"], bins=40, color="skyblue", edgecolor="black", alpha=0.7, label="Simulations")


plt.title("Sample size distribution  - Equalized Odds (fair)")
plt.xlabel("Sample size")
plt.ylabel("Frecuency")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()